In [1]:
import os
import time
from pathlib import Path
import traceback
import json
import sys
import argparse

from langchain.vectorstores import FAISS
#from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
import json

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

#from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
#from langchain_openai import ChatOpenAI

#from langchain import PromptTemplate 
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
#from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.callbacks import get_openai_callback
import APIKEY
os.environ["OPENAI_API_KEY"] = APIKEY.API_KEY_SERVICE_OPENAI

In [2]:
from langsmith import Client

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Agent_test1"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = APIKEY.LANGCHAIN_API_KEY  # Update to your API key

## Create the agent

In [26]:
#input_doc_pth =r'C:\Users\USER\Desktop\llma\LLM_Playing\pyinstaller_bg\doc_faiss\QA\TRM_M463_M467_pypdf'
input_doc_pth = r'D:\nu_QA_data\TRM_M463_M467_pypdf_0308'
gptmodel = 'gpt-4-1106-preview' #'gpt-3.5-turbo-0125'
mmr_num = 8
chip_type = 'm460'

#system_template = """Use the following pieces of context and chat history to answer the question at the end. The context is Nuvoton """+chip_type+""" Series Technical Reference Manual.
#If you don't know the answer or the question has nothing to do with technical, don't try to make up an answer.
#"""

#system_template = """You are a technical engineer to answer chip or math relative question at the end.
#You can access two tools, first is searching Nuvoton """+chip_type+""" Series Technical Reference Manual and the second tool is calculator.
#Organize the context from searching Technical Reference Manual or calculator to finish the answer.
#If you don't know the answer or the question has nothing to do with technical, don't try to make up an answer.
#"""

system_template = """You are a technical engineer to answer chip or math relative question at the end.
You can access three tools, first is searching Nuvoton """+chip_type+""" Series Technical Reference Manual.
If you can't get enough infomation, use the second tool, searching tables in Technical Reference Manual.
The third tool is calculator.
Organize the context from these tools to finish the answer.
If you don't know the answer or the question has nothing to do with technical, don't try to make up an answer.
"""


messages =[("system", system_template),
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name='agent_scratchpad')]

qa_prompt = ChatPromptTemplate.from_messages(messages)

llm = ChatOpenAI(temperature=0, model=gptmodel)
    
# choose your embeddings model
embeddings = OpenAIEmbeddings()
# FAISS  
vectorstore=FAISS.load_local(input_doc_pth, embeddings)
retriever_vec=vectorstore.as_retriever(
search_type="mmr", # Also test "similarity"
search_kwargs={"k": mmr_num})

# Add table retriver
input_doc_table_pth = r'D:\nu_QA_data\TRM_M463_M467_table_0308'
vectorstore_table=FAISS.load_local(input_doc_table_pth, embeddings)
retriever_vec_table=vectorstore_table.as_retriever(
search_type="mmr", # Also test "similarity"
search_kwargs={"k": 8})



In [27]:
from langchain.tools.retriever import create_retriever_tool

tool_trm_search = create_retriever_tool(
    retriever=retriever_vec,
    name = "search_{}_technical_reference_manual".format(chip_type),
    description = "Searches and returns detail from the {} technical reference manual.".format(chip_type),
)

tool_trm_table_search = create_retriever_tool(
    retriever=retriever_vec_table,
    name = "search_{}_technical_reference_manual_tables".format(chip_type),
    description = "Searches and returns detail from the {} tbales in technical reference manual.".format(chip_type),
)

from langchain.tools import BaseTool
class EvaluateMathExpression(BaseTool):
    name = "Calculator"
    description = "Use this tool to evaluate a math expression."
    def _run(self, expr: str):
        return eval(expr)
    def _arun(self, query: str):
        raise NotImplementedError("Async not supported")

tools = [tool_trm_search, tool_trm_table_search, EvaluateMathExpression()]

In [28]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

def create_agent():
    agent = create_openai_tools_agent(llm, tools, qa_prompt)
    return AgentExecutor(agent=agent, tools=tools)

agent_executor = create_agent()
chat_history_list = []

## Start the agent

In [29]:
result = agent_executor.invoke({"input": "What is APB and PCLK Maximum Speed ( MHz) of Turbo mode Power Level 0 (PL0)?",
                               "chat_history": chat_history_list})

In [25]:
print(result)
chat_history_list.append(HumanMessage(content = result['input']))
chat_history_list.append(AIMessage(content = result['output']))

{'input': 'What is APB and PCLK Maximum Speed ( MHz) of Turbo mode Power Level 0 (PL0)?', 'chat_history': [], 'output': 'The APB and PCLK maximum speed for Turbo mode Power Level 0 (PL0) is not explicitly stated in the provided documents. However, the closest information available is for the Normal mode, where the HCLK maximum speed is 180 MHz and the APB and PCLK maximum speed is 90 MHz at 1.20V. Since Turbo mode typically operates at a higher performance level than Normal mode, it is reasonable to infer that the APB and PCLK maximum speed in Turbo mode PL0 would be equal to or greater than the speed in Normal mode. However, without explicit documentation, I cannot provide a definitive answer for the Turbo mode PL0 speeds.'}


In [11]:
result = agent_executor.invoke({"input": "What is my name?",
                               "chat_history": chat_history_list})

In [13]:
print(result)
chat_history_list.append(HumanMessage(content = result['input']))
chat_history_list.append(AIMessage(content = result['output']))

{'input': 'What is my name?', 'chat_history': [HumanMessage(content='Hi my name is Gary'), AIMessage(content='Hello Gary! How can I assist you today?')], 'output': 'Your name is Gary. How can I assist you today, Gary?'}


In [14]:
result = agent_executor.invoke({"input": "What is 3 * 2 - 1.001 + 3",
                               "chat_history": chat_history_list})

In [15]:
print(result)
chat_history_list.append(HumanMessage(content = result['input']))
chat_history_list.append(AIMessage(content = result['output']))

{'input': 'What is 3 * 2 - 1.001 + 3', 'chat_history': [HumanMessage(content='Hi my name is Gary'), AIMessage(content='Hello Gary! How can I assist you today?'), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Gary. How can I assist you today, Gary?')], 'output': 'The result of the expression 3 * 2 - 1.001 + 3 is approximately 7.999. If you have any more questions or need further assistance, feel free to ask!'}


In [16]:
result = agent_executor.invoke({"input": "what is the max cpu speed of m460?",
                               "chat_history": chat_history_list})

## New conversation

In [43]:
chat_history_list = []

In [44]:
result = agent_executor.invoke({"input": "what is the max cpu speed of m460?",
                               "chat_history": chat_history_list})

In [45]:
print(result)

{'input': 'what is the max cpu speed of m460?', 'chat_history': [], 'output': 'The maximum CPU speed of the Nuvoton M460 series is 200 MHz. This information is derived from the fact that the on-chip PLL (Phase-Locked Loop) can go up to 400 MHz, allowing CPU operation up to the maximum CPU frequency without the need for a high-frequency crystal. However, the specific maximum CPU speed is not directly stated in the provided documents. Typically, the CPU speed is half the PLL frequency due to the internal division, which would suggest a 200 MHz maximum CPU speed for the M460 series.'}


## Config evaluators

In [19]:
DATASET_NAME = "ds_m460_trm_agent"
client = Client()

In [20]:
import re
from typing import Any, Optional
from langchain_core.prompts import PromptTemplate
from langchain.evaluation import StringEvaluator

class GradeEvaluator(StringEvaluator):
    """An LLM-based relevance evaluator."""
    def __init__(self):
        llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

        template = """You are a technical professor responsible for checking the correctness of technical issues and 
grading the students' answers to questions. You are given a question, the student's answer, and the true answer. 
You are asked to score the student's answer on a scale from 0 to 100.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: score here between 0 to 100

Grade the student answers based ONLY on their factual accuracy and 
note if the values of the any parameters are correct.
Ignore differences in punctuation and phrasing between the student answer and true answer. 
It is OK if the student answer contains more information than the true answer, 
as long as it does not contain any conflicting statements. Begin!
QUESTION:{input}
STUDENT ANSWER:{prediction}
TRUE ANSWER:{reference}
GRADE:
"""
        self.eval_chain = PromptTemplate.from_template(template) | llm

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return True

    @property
    def evaluation_name(self) -> str:
        return "scored_relevance"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain.invoke(
            {"input": input, "prediction": prediction, "reference": reference}, kwargs
        )
        score = evaluator_result.content
        score = re.search(r"\d+", score).group(0)
        if score is not None:
            score = float(score.strip()) / 100.0
        return {"score": score}

In [21]:
from langchain.evaluation import EvaluatorType
from langchain.smith import RunEvalConfig
from langchain_core.prompts.prompt import PromptTemplate

#_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
#You are grading the following question:
#{query}
#Here is the real answer:
#{answer}
#You are grading the following predicted answer:
#{result}
#Respond with CORRECT or INCORRECT:
#Grade:
#"""
#PROMPT = PromptTemplate(input_variables=['result', 'query', 'answer'], template=_PROMPT_TEMPLATE)

evaluation_config = RunEvalConfig(
    input_key='input',
    prediction_key='output',
    evaluators=[
        #EvaluatorType.EMBEDDING_DISTANCE,
        RunEvalConfig.QA(llm=ChatOpenAI(temperature=0, model='gpt-4-0125-preview')),
        RunEvalConfig.LabeledScoreString(
            {
               "accuracy": """
Score 1: The answer is completely unrelated to the reference.
Score 3: The answer has minor relevance but does not align with the reference.
Score 5: The answer has moderate relevance but contains inaccuracies.
Score 7: The answer aligns with the reference but has minor errors or omissions.
Score 10: The answer is completely accurate and aligns perfectly with the reference."""
            },
            normalize_by=10,
            llm=ChatOpenAI(temperature=0, model='gpt-4-0125-preview')
        ),
    ],
    # You can add custom StringEvaluator or RunEvaluator objects here as well, which will automatically be
    # applied to each prediction. Check out the docs for examples.
    custom_evaluators=[GradeEvaluator()]
)

## Run the agent and evaluators

In [22]:
import functools

from langchain.smith import arun_on_dataset, run_on_dataset

chain_results = run_on_dataset(
    dataset_name=DATASET_NAME,
    llm_or_chain_factory=create_agent(),
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"rag2-agent-qa-2",
    # Project metadata communicates the experiment parameters,
    # Useful for reviewing the test results
    project_metadata={
        "env": "agent.ipynb",
        "version": "0.0.1",
        "model": "gpt-4-1106-preview",
        "prompt": "QA_agent_0308",
        "input_doc": "TRM_M463_M467_pypdf, TRM_M463_M467_table_0308",
        "langchain_type": "create_openai_tools_agent"
    },
)

View the evaluation results for project 'rag2-agent-qa-2' at:
https://smith.langchain.com/o/a2f265bc-195d-4980-affd-877d4fb55720/datasets/9990cff8-6b80-4d9c-9143-56b6be7410af/compare?selectedSessions=cc64d4be-e387-42f9-944e-d7b942b1a897

View all tests for Dataset ds_m460_trm_agent at:
https://smith.langchain.com/o/a2f265bc-195d-4980-affd-877d4fb55720/datasets/9990cff8-6b80-4d9c-9143-56b6be7410af
[------------------------------------------------->] 24/24

,feedback.correctness,feedback.score_string:accuracy,feedback.scored_relevance,error,execution_time,run_id
count,24.000000,24.000000,24.000000,0,24.000000,24
unique,NaN,NaN,NaN,0,NaN,24
top,NaN,NaN,NaN,NaN,NaN,e6152069-f696-47f5-8878-a97eba29cb10
freq,NaN,NaN,NaN,NaN,NaN,1
mean,0.750000,0.804167,0.855417,NaN,21.514409,NaN
std,0.442326,0.256191,0.247790,NaN,10.573627,NaN
min,0.000000,0.300000,0.000000,NaN,10.699575,NaN
25%,0.750000,0.700000,0.850000,NaN,13.240505,NaN
50%,1.000000,1.000000,0.950000,NaN,18.591372,NaN
75%,1.000000,1.000000,1.000000,NaN,25.078500,NaN


## Copy langsmith dataset and edit examples

In [33]:
old_dataset_name = 'ds_m460_trm'
examples = client.list_examples(dataset_name=old_dataset_name)

In [34]:
example_inputs = []
for x in examples:
    data_tuple = (x.inputs['question'], x.outputs['output'])
    #print(data_tuple)
    example_inputs.append(data_tuple)

In [35]:
print(len(example_inputs))

24


In [36]:
client = Client()
dataset_name = "ds_m460_trm_agent"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name, description="ds_m460_trm, agent style, only input & output",
)
for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"input": input_prompt},
        outputs={"output": output_answer},
        dataset_id=dataset.id,
    )